In [1]:
import pandas as pd
import numpy as np
from automation_process_dataV2 import united_functions
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [2]:
df_rating = pd.read_parquet('../../data/raw/data_model/ratings_train05.parquet')#Carga de datos

In [3]:
# muestrear 0.2% de regitros
_, subset_rating = train_test_split(df_rating, test_size=0.002, random_state=111)

In [5]:
# Guardar muestra para aplicarla en la app final
subset_rating.to_parquet('../../data/raw/data_model/final_data_used/subset_splitV2.parquet')

In [6]:
subset_rating.reset_index(inplace=True, drop=True)

In [7]:
subset_rating.shape

(2001, 6)

In [8]:
df_processed, list_user_movies= united_functions(subset_rating)# Script de procesamiento de datos y retorno de titulo con id

INICIANDO PROCESAMIENTO DE DATOS...
- Eliminacion de nulos..
- Eliminando variables no necesarias...
- Extrayendo año del titulo...
- Almacenando en variable copia del df original...
- Extraemos los generos de cada pelicula...
- Eliminamos variables no necesarias...
- Codificamos los generos y normalizamos para su correcto formato...
FIN DE PROCESAMIENTO DE DATOS


In [9]:
df_processed.shape

(2001, 21)

In [10]:
# Guardar resutado de procesamiento para aplicarla en la app final
df_processed.to_csv('../../data/raw/data_model/final_data_used/process_splitV2.csv', index=False)
list_user_movies.to_csv('../../data/raw/data_model/final_data_used/list_user_moviesV2.csv', index=False)

In [11]:
# Diccionario de list_user_movies
dicc_user_movies = {'userId':list_user_movies['userId'], 'movieId':list_user_movies['movieId'], 'title':list_user_movies['title']}

Similitud del coseno (compara cada pelicula con todas las encontradas)

In [12]:
simil_cosine=cosine_similarity(df_processed)# calculo de similitus del coseno

In [13]:
# Rellenamos la diagonal con NaN y convertimos a dataframe
np.fill_diagonal(simil_cosine, np.nan)
df_similCosine = pd.DataFrame(simil_cosine)

In [14]:
df_similCosine

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,NaN,0.843428,0.594415,0.286990,0.533208,0.482399,0.701530,0.589643,0.847257,0.496401,...,0.390592,0.504716,0.294600,0.634479,0.572945,0.479707,0.307607,0.313744,0.462326,0.785469
1,0.843428,NaN,0.719865,0.306876,0.632761,0.289994,0.820801,0.346512,0.990637,0.588075,...,0.412079,0.534662,0.321590,0.732730,0.367171,0.196818,0.327712,0.333956,0.494644,0.648961
2,0.594415,0.719865,NaN,0.160108,0.498939,0.144188,0.598924,0.180433,0.715948,0.420044,...,0.224959,0.287931,0.156036,0.531433,0.206993,0.127863,0.173139,0.176965,0.257567,0.478600
3,0.286990,0.306876,0.160108,NaN,0.162507,0.542132,0.261890,0.664622,0.321951,0.548450,...,0.461071,0.618625,0.654933,0.250074,0.414267,0.123378,0.660698,0.401275,0.623657,0.407076
4,0.533208,0.632761,0.498939,0.162507,NaN,0.141761,0.538549,0.182907,0.643354,0.581969,...,0.234755,0.298035,0.150796,0.486848,0.220044,0.146017,0.177126,0.413682,0.261099,0.439122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,0.479707,0.196818,0.127863,0.123378,0.146017,0.756744,0.197121,0.409122,0.236164,0.115356,...,0.188627,0.235643,0.102228,0.449691,0.859768,NaN,0.136732,0.377204,0.197704,0.637909
1997,0.307607,0.327712,0.173139,0.660698,0.177126,0.393708,0.282173,0.482258,0.346357,0.610060,...,0.508940,0.348833,0.991506,0.270630,0.243079,0.136732,NaN,0.442864,0.319372,0.233598
1998,0.313744,0.333956,0.176965,0.401275,0.413682,0.367989,0.288160,0.228083,0.353578,0.377847,...,0.275743,0.356057,0.433018,0.684115,0.434099,0.377204,0.442864,NaN,0.325587,0.238969
1999,0.462326,0.494644,0.257567,0.623657,0.261099,0.280386,0.421547,0.337042,0.518347,0.587063,...,0.739305,0.991948,0.309982,0.402248,0.361038,0.197704,0.319372,0.325587,NaN,0.346023


In [15]:
# Guardar matriz similitudes coseno
df_similCosine.to_csv('cosine_csv/df_similCosineV2.csv', index=False)

Logica que implementa la recomendacion en base a un usuario y pelicula vista

In [16]:
def comprobar_existencia(diccionario, movie_id, user_id):    
    for i in range(len(diccionario['userId'])):
        if diccionario['userId'].iloc[i] == user_id and diccionario['movieId'].iloc[i] == movie_id:
            return True
    return False

In [17]:
def obtener_index_movieId(diccionario, movie_id):
    for i in range(len(diccionario['movieId'])):
        if diccionario['movieId'].iloc[i]==movie_id:
            index = i
            break
    return index

In [18]:
def get_title_from_movieId(user_id ,diccionario):
    index = diccionario['userId'].loc[diccionario['userId'] == user_id].index
    # Obtén los títulos de las películas
    titles_saws = diccionario['title'].loc[index]
    return titles_saws

In [19]:
# Funcion de recomendacion para un usuario y pelicula
def get_recommendations(user_id, movie_id,dicc_user_movies=dicc_user_movies, simil_cosine=df_similCosine, threshold=0.7):
    #Comprobamos que el usuario para la pelicula dada existe
    comprobar_user_movie_exist=comprobar_existencia(dicc_user_movies, movie_id, user_id)

    if comprobar_user_movie_exist:
       #Obtener indice de la pelicula dada
        index=obtener_index_movieId(dicc_user_movies, movie_id)

        # Obtén la lista de similitudes del coseno para una película y usuario dada
        cosine_sim_list = simil_cosine.iloc[index]
        
        # Ordena las películas en función de la similitud del coseno
        most_similar_movies_sorted=cosine_sim_list.sort_values(ascending=False)

        # Obtiene los índices de las 5 películas más similares superior al umbral 
        most_similar_movies = most_similar_movies_sorted[most_similar_movies_sorted > threshold][:5].index

        # Obtiene los títulos de las películas más similares
        most_similar_movie_titles = list(set([dicc_user_movies['title'][id] for id in most_similar_movies if id in dicc_user_movies['title']]))
        
        # Obtener los titulos de todas las peliculas vistas por el usuario
        titles_saws = get_title_from_movieId(user_id ,dicc_user_movies)
        titles_saws = list(titles_saws)
        
        # Pelicula actual pasada por parametros (title)
        actual_title = dicc_user_movies['movieId'].loc[dicc_user_movies['movieId'] == movie_id].index[0]

        print('Porque has visto:',dicc_user_movies['title'].loc[actual_title])
        
        # Filtra las películas recomendadas para excluir las que el usuario ya ha visto
        recommended_movies = [movie for movie in most_similar_movie_titles if movie not in titles_saws]

        # Si las peliculas se devuelven vacias se retorna la condicion
        if not recommended_movies:
            return 'No se encontro una pelicula lo suficientemente significativa para recomendar'
        print('Recomendamos:')
        return recommended_movies
    else: 
        return 'No se encontro usuario para pelicula dada'
        

In [21]:
result_recommendation=get_recommendations(47985,33004)#userId, movieId
result_recommendation

Porque has visto: Hitchhiker's Guide to the Galaxy, The
Recomendamos:


['Galaxy Quest',
 "Bill & Ted's Excellent Adventure",
 'Back to the Future Part II']

Podemos ver que para la pelicula Hitchhiker's Guide to the Galaxy, The (cuyo genero es comedia/ciencia ficcion lanzada en el 2005), 

se recomiendan tres peliculas de comedia/ciencia, ambas lanzadas en años similares.

In [20]:
list_user_movies.tail(10)

,userId,movieId,rating,title
1991,47985.0,33004,4.5,"Hitchhiker's Guide to the Galaxy, The"
1992,116941.0,5401,4.0,Undercover Brother
1993,7405.0,2826,2.0,"13th Warrior, The"
1994,87208.0,2762,5.0,"Sixth Sense, The"
1995,53360.0,1617,5.0,L.A. Confidential
1996,83373.0,2206,3.5,Suspicion
1997,104655.0,2193,3.0,Willow
1998,50541.0,3081,3.5,Sleepy Hollow
1999,11984.0,700,3.0,Angus
2000,27783.0,1245,5.0,Miller's Crossing


In [17]:
# Buscar usuarios 
def movies_peruser(idUser):    
    print(list_user_movies[list_user_movies['userId']==idUser])

In [24]:
movies_peruser(136540)

         userId  movieId  rating             title
19995  136540.0       36     3.0  Dead Man Walking
